In [1]:
!pip install jdaviz

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 31.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/530.3 kB ? eta -:--:--
   ---------------------------------------- 530.3/530.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/930.9 kB ? eta -:--:--
   --------------------------------------- 930.9/930.9 kB 41.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 查詢 NED 提供的星系資料

In [1]:
# 引入 ipac.ned 模組中的 Ned 類別，用於查詢 NED 提供的星系資料
from astroquery.ipac.ned import Ned

# 定義要查詢的星系名稱
galaxy_name = 'M104'

# 使用 Ned 物件的 query_object() 方法來查詢星系的基本資訊
ned_query_result_table = Ned.query_object(galaxy_name)
print(f'{galaxy_name} 星系基本資訊:')
print(ned_query_result_table)

# 使用 Ned 物件的 get_images_async() 方法來取得星系的影像資料
ned_images = Ned.get_images_async(galaxy_name)

# 顯示第一個影像 FITS 檔案的資訊，並取出其影像的像素值資料
print('影像資料 FITS 檔案資訊:')
ned_images[0].get_fits().info()
print('影像的像素資料:')
ned_image_data = ned_images[0].get_fits()[0].data
print(ned_image_data)

# 使用 Ned 物件的 get_spectra_async() 方法來取得星系的光譜資料
ned_spectra = Ned.get_spectra_async(galaxy_name)

# 顯示最後一個光譜 FITS 檔案的資訊，並取出其光譜資料
print('光譜資料 FITS 檔案資訊:')
ned_spectra[-1].get_fits().info()
print('光譜資料:')
ned_spectrum_data = ned_spectra[-1].get_fits()[1].data
print(ned_spectrum_data)


M104 星系基本資訊:
No. Object Name     RA     ... Redshift Points Diameter Points Associations
                 degrees   ...                                             
--- ----------- ---------- ... --------------- --------------- ------------
  1 MESSIER 104  189.99763 ...              51               7            0
影像資料 FITS 檔案資訊:
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     106   (424, 424)   int16   
影像的像素資料:
[[5313 5392 5392 ... 5436 5121 5121]
 [5156 5550 5235 ... 5436 5436 5279]
 [5471 5550 5865 ... 5751 5751 5909]
 ...
 [5361 5913 5598 ... 6378 5748 5748]
 [5361 5598 5283 ... 6378 5748 5748]
 [5283 5598 5913 ... 5984 5669 6142]]
光譜資料 FITS 檔案資訊:
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      46   ()      
  1  wrapped       1 BinTableHDU    380   208R x 5C   [1D, 1J, 1J, 1D, 1D]   

## 查詢 SDSS (Sloan Digital Sky Survey)  提供的星系資料

In [ ]:
from astroquery.mast import Observations
from jdaviz import Specviz
import os

# 查詢資料
obs_table = Observations.query_criteria(
    target_name='J0100+2802',
    obs_collection='JWST',
    dataproduct_type='SPECTRUM'
)

if len(obs_table) == 0:
    print("無法找到符合條件的觀測資料")
    exit()

# 轉換成 Pandas DataFrame
obs_table_df = obs_table.to_pandas()

# 嘗試下載資料
try:
    product_list_table = Observations.get_product_list(obs_table[0])
    product_list_table_df = product_list_table.to_pandas()

    product_list_table_df = product_list_table_df[
        (product_list_table_df['productType'] == 'SCIENCE') &
        (product_list_table_df['description'].str.contains('1D extracted spectrum'))
    ]

    if len(product_list_table_df) == 0:
        print("無法找到符合條件的光譜資料")
        exit()

    product_list_table_index = product_list_table_df.index[-1]
    downloaded_product_table = Observations.download_products(
        product_list_table[product_list_table_index], extension='fits'
    )

    downloaded_fits_filepath = downloaded_product_table[0]['Local Path']

    # 檢查檔案是否存在
    if not os.path.exists(downloaded_fits_filepath):
        print(f"檔案未找到：{downloaded_fits_filepath}")
        exit()

    # 載入光譜資料
    specviz = Specviz()
    specviz.load_data(downloaded_fits_filepath)
    specviz.show()

except Exception as e:
    print(f"資料處理時出現錯誤：{e}")


檔案未找到：.\mastDownload\JWST\jw01243-o001_s03482_nircam_f356w-grismr\jw01243-o001_s03482_nircam_f356w-grismr_x1d.fits
資料處理時出現錯誤：The input file '.\mastDownload\JWST\jw01243-o001_s03482_nircam_f356w-grismr\jw01243-o001_s03482_nircam_f356w-grismr_x1d.fits' cannot be parsed as a URL or URI, and no existing local file is available at this path.


: 

In [1]:
# 引入 mast 模組中的 Observations 類別，用於查詢 MAST 提供的星系觀測資料
from astroquery.mast import Observations
# 引入 astropy 中的 fits 子套件，用於 FITS 檔案的操作
from astropy.io import fits

# 定義一個用於取得 MAST 提供的星系影像和光譜資料的函式
def fetch_galaxy_data_from_mast(galaxy_name, telescope='HST', data_type='IMAGE', observation_index=0, product_index=0):
    try:
        # 檢查 telescope 參數是否為有效值
        if telescope not in ['HST', 'JWST']:
            raise ValueError('telescope 參數必須是 "HST" 或 "JWST" 其中之一')
        
        # 檢查 data_type 參數是否為有效值
        if data_type not in ['IMAGE', 'SPECTRUM']:
            raise ValueError('data_type 參數必須是 "IMAGE" 或 "SPECTRUM" 其中之一')
        
        # 使用 Observations 物件的 query_criteria() 方法查詢資料
        obs_table = Observations.query_criteria(objectname=galaxy_name, obs_collection=telescope, dataproduct_type=data_type)
        print(f'找到 {len(obs_table)} 筆觀測，請查看回傳的 obs_table 變數')
        
        # 若沒有觀測資料，則返回 None
        if len(obs_table) == 0:
            return obs_table, None, None

        # 檢查 observation_index 參數是否在有效範圍內
        if observation_index >= len(obs_table):
            print("輸入的 observation_index 參數超出 obs_table 的觀測筆數，請修改")
            return obs_table, None, None

        # 取得產品列表
        product_list_table = Observations.get_product_list(obs_table[observation_index])
        print(f'你選擇 obs_table 中的第 {observation_index + 1} 筆觀測，該觀測有 {len(product_list_table)} 筆觀測資料')
        
        # 檢查 product_index 參數是否在有效範圍內
        if product_index >= len(product_list_table):
            print("輸入的 product_index 參數超出 product_list_table 的觀測資料筆數，請修改")
            return obs_table, product_list_table, None
        
        # 下載指定的觀測資料
        print(f'正在下載 product_list_table 中的第 {product_index + 1} 筆觀測資料，請稍後......')
        downloaded_product_table = Observations.download_products(product_list_table[product_index], extension='fits')
        print("下載成功！請查看回傳的 downloaded_product_table 變數")
        return obs_table, product_list_table, downloaded_product_table
    
    except Exception as e:
        print(f'星系資料取得失敗，錯誤訊息：{e}')
        return None, None, None

# 設定要查詢的星系名稱
galaxy_name = 'M104'

# 呼叫 fetch_galaxy_data_from_mast() 函式來查詢並下載星系的觀測資料
obs_table, product_list_table, downloaded_product_table = fetch_galaxy_data_from_mast(
    galaxy_name, observation_index=14, product_index=9
)

# 如果成功下載觀測資料，則顯示下載的 FITS 檔案資訊
if downloaded_product_table:
    filename = downloaded_product_table[0]['Local Path']
    hdu_list = fits.open(filename)
    hdu_list.info()


找到 507 筆觀測，請查看回傳的 obs_table 變數
你選擇 obs_table 中的第 15 筆觀測，該觀測有 13 筆觀測資料
正在下載 product_list_table 中的第 10 筆觀測資料，請稍後......
下載成功！請查看回傳的 downloaded_product_table 變數
Filename: .\mastDownload\HST\w0tr0e03t\w0tr0e03t_d0f.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     194   (800, 800, 4)   int16   
  1  w0tr0e03t.d0h.tab    1 TableHDU       158   4R x 37C   [D25.16, D25.16, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, A4, E15.7, I11, I11, D25.16, D25.16, A8, A8, I11, E15.7, E15.7, E15.7, E15.7, E15.7, E15.7, I11, I11, I11, I11, I11, I11, I11, A24, E15.7, E15.7, E15.7, E15.7]   


查詢和下載 James Webb 太空望遠鏡 (JWST) 的觀測資料，並使用 jdaviz 的 Imviz 來視覺化 FITS 影像檔案。

In [2]:
# 引入 astroquery.mast 模組中的 Observations 類別
# 用於查詢 MAST 提供的星系觀測資料
from astroquery.mast import Observations
# 從 jdaviz 套件中引入 Imviz 類別，用來視覺化天文影像資料
from jdaviz import Imviz

# 使用 Observations 類別的 query_criteria() 方法來查詢目標天體的影像資料
# 查詢 JWST 觀測到的星系團 Abell 2744 的資料
obs_table = Observations.query_criteria(
    target_name="Abell2744", 
    obs_collection="JWST", 
    dataproduct_type="IMAGE"
)

# 將傳回的查詢結果 (astropy 的 table 物件) 轉換成 Pandas dataframe，以便進行資料篩選
obs_table_df = obs_table.to_pandas()

# 進行資料篩選：過濾曝光時間超過 5000 秒，且濾鏡為 F444W 的資料
obs_table_df = obs_table_df[(obs_table_df['t_exptime'] > 5000) & (obs_table_df['filters'] == 'F444W')]

# 選取篩選後的第一筆資料的索引，然後使用該索引取得相應的 product list
obs_table_index = obs_table_df.index[0]
product_list_table = Observations.get_product_list(obs_table[obs_table_index])

# 將 product_list_table (astropy 的 table 物件) 轉換成 Pandas dataframe
product_list_table_df = product_list_table.to_pandas()

# 進行資料篩選：選擇 productType 為 'SCIENCE' 且 description 包含 'rectified 2D image' 的觀測資料
product_list_table_df = product_list_table_df[
    (product_list_table_df['productType'] == 'SCIENCE') &
    (product_list_table_df['description'].str.contains('rectified 2D image'))
]

# 取得篩選後最後一筆資料的索引，然後下載相應的 FITS 檔案
product_list_table_index = product_list_table_df.index[-1]
downloaded_product_table = Observations.download_products(
    product_list_table[product_list_table_index],
    extension='fits'
)

# 取得下載的觀測資料 FITS 檔案路徑
downloaded_fits_filepath = downloaded_product_table[0]['Local Path']

# 使用 Imviz 物件的 load_data() 方法載入 FITS 檔案，並顯示影像資料
imviz = Imviz()
imviz.load_data(downloaded_fits_filepath)
imviz.show()


Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v4.0.0/imviz/index.html', events=['cal…

In [ ]:
from astroquery.mast import Observations
from jdaviz import Specviz
import os
from astropy.io import fits

# 查詢 JWST 的光譜資料
obs_table = Observations.query_criteria(
    target_name='J0100+2802',
    obs_collection='JWST',
    dataproduct_type='SPECTRUM'
)
obs_table_df = obs_table.to_pandas()

# 篩選 Quasar 的資料
obs_table_df = obs_table_df[obs_table_df['target_classification'].fillna('').str.contains('Quasars')]
obs_table_index = obs_table_df.index[0]
product_list_table = Observations.get_product_list(obs_table[obs_table_index])
product_list_table_df = product_list_table.to_pandas()

# 篩選 1D 光譜資料
product_list_table_df = product_list_table_df[
    (product_list_table_df['productType'] == 'SCIENCE') &
    (product_list_table_df['description'].str.contains('1D extracted spectrum'))
]
product_list_table_index = product_list_table_df.index[-1]

# 下載 FITS 檔案
downloaded_product_table = Observations.download_products(
    product_list_table[product_list_table_index], extension='fits'
)
downloaded_fits_filepath = os.path.abspath(downloaded_product_table[0]['Local Path'])

# 檢查檔案是否存在
if os.path.exists(downloaded_fits_filepath):
    print(f"成功下載檔案：{downloaded_fits_filepath}")
else:
    print("下載失敗，請檢查路徑或重新下載。")
    exit()

# 檢查 FITS 檔案是否可讀取
try:
    with fits.open(downloaded_fits_filepath) as hdul:
        hdul.info()
except Exception as e:
    print(f"無法讀取 FITS 檔案：{e}")
    exit()

# 使用 Specviz 進行光譜分析
try:
    specviz = Specviz()
    specviz.load_data(downloaded_fits_filepath)
    specviz.show()
except Exception as e:
    print(f"載入光譜資料時出現錯誤：{e}")


下載失敗，請檢查路徑或重新下載。
無法讀取 FITS 檔案：[Errno 2] No such file or directory: 'd:\\github\\obsidian-academic-research\\python在星空作畫\\mastDownload\\JWST\\jw01243-o001_s03482_nircam_f356w-grismr\\jw01243-o001_s03482_nircam_f356w-grismr_x1d.fits'
載入光譜資料時出現錯誤：URI d:\github\obsidian-academic-research\python在星空作畫\mastDownload\JWST\jw01243-o001_s03482_nircam_f356w-grismr\jw01243-o001_s03482_nircam_f356w-grismr_x1d.fits with scheme d is not currently supported.


: 

In [ ]:
!pip uninstall bokeh
!pip install bokeh --upgrade